# Day19_2: AI 에이전트 (Agent) 구축하기 (정답)

## 학습 목표

**Part 1: 기초**
1. 표준 LLM 및 RAG 시스템과 AI 에이전트의 차이점 이해하기
2. ReAct (Reason + Act) 프레임워크 개념 이해하기
3. 에이전트의 핵심 구성 요소(LLM, 도구, 프롬프트) 이해하기
4. LangChain @tool 데코레이터로 도구 만들기
5. 에이전트 실행기(Agent Executor) 이해하기

**Part 2: 심화**
1. 여러 도구를 활용하는 멀티-툴 에이전트 구축하기
2. 대화 메모리(Memory) 통합하기
3. RAG 도구 + 웹 검색 도구 결합하기
4. 뉴스 분석가 에이전트 실습하기

---

## 실습 환경 설정

In [ ]:
# 필수 라이브러리 임포트
import os
import datetime
from dotenv import load_dotenv

# LangChain 관련
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.agents import tool, create_openai_tools_agent, AgentExecutor
from langchain.tools import Tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory
from langchain_chroma import Chroma
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 환경 변수 로드
load_dotenv()

# LLM 및 임베딩 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

print("환경 설정 완료!")

In [ ]:
# 본문 예제에서 사용한 도구들 정의

@tool
def get_current_time(ignored_input: str = "") -> str:
    """
    현재 한국 시간을 'YYYY-MM-DD HH:MM:SS' 형식으로 반환합니다.
    현재 시각, 지금 몇 시, 오늘 날짜 등의 질문에 사용하세요.
    """
    return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@tool
def calculator(expression: str) -> str:
    """
    수학 계산을 수행합니다. 덧셈, 뺄셈, 곱셈, 나눗셈을 지원합니다.
    입력: 계산식 문자열 (예: "2 + 3 * 4")
    출력: 계산 결과
    """
    try:
        allowed_chars = set('0123456789+-*/().  ')
        if not all(c in allowed_chars for c in expression):
            return "오류: 허용되지 않는 문자가 포함되어 있습니다."
        result = eval(expression)
        return f"{expression} = {result}"
    except Exception as e:
        return f"계산 오류: {str(e)}"

print("기본 도구 생성 완료!")

In [ ]:
# 뉴스 벡터 DB 생성 (퀴즈 풀이용)
news_articles = [
    {
        "title": "데이터브릭스, 차세대 AI 거버넌스 프레임워크 '유니티 카탈로그 2.0' 발표",
        "content": "2025년 1월, 데이터 및 AI 기업 데이터브릭스는 차세대 데이터 거버넌스 프레임워크인 '유니티 카탈로그 2.0'을 공개했다. 이 프레임워크는 데이터, AI 모델, 관련 파이프라인 전체에 걸쳐 통합된 보안 및 거버넌스를 제공한다.",
        "source": "데이터 이코노미",
        "date": "2025-01-20"
    },
    {
        "title": "AI 옵저버빌리티 플랫폼 '뉴럴와처', 2억 달러 투자 유치",
        "content": "AI 모델의 운영 상태를 실시간으로 감시하는 'AI 옵저버빌리티' 분야가 급성장하고 있다. 스타트업 '뉴럴와처'는 시리즈 C 펀딩에서 2억 달러를 유치했다.",
        "source": "AI 스타트업 위클리",
        "date": "2025-01-22"
    },
    {
        "title": "구글 클라우드, '버텍스 AI 에이전트 빌더' 정식 출시",
        "content": "구글 클라우드는 버텍스 AI에 '에이전트 빌더' 기능을 정식으로 추가했다. 코딩 경험이 없는 사용자도 AI 에이전트를 구축할 수 있다.",
        "source": "클라우드 인사이트",
        "date": "2025-01-25"
    }
]

news_documents = [
    Document(
        page_content=article["content"],
        metadata={"title": article["title"], "source": article["source"], "date": article["date"]}
    )
    for article in news_articles
]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(news_documents)

news_vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding_model,
    collection_name="news_for_agent_answer"
)

retriever = news_vectordb.as_retriever(search_kwargs={"k": 3})

print("벡터 DB 생성 완료!")

---

## 퀴즈 정답

---

### Q1. LLM vs RAG vs 에이전트 비교하기

**문제**: 표준 LLM, RAG, AI 에이전트의 차이점을 각각 한 문장으로 설명하고, 각각에 적합한 사용 사례를 하나씩 제시하세요.

In [ ]:
# 정답
answer_q1 = """
=== LLM vs RAG vs 에이전트 비교 ===

1. 표준 LLM (Large Language Model)
   - 정의: 학습된 지식을 바탕으로 질문에 답변하는 언어 모델
   - 한 문장 설명: 내부 기억(학습 데이터)에만 의존하여 답변을 생성합니다.
   - 적합한 사례: 일반적인 지식 질문 ("파이썬이 뭐야?", "광합성 과정 설명해줘")

2. RAG (Retrieval-Augmented Generation)
   - 정의: 외부 문서를 검색하여 LLM에게 제공하는 시스템
   - 한 문장 설명: 외부 지식 베이스에서 관련 문서를 검색하여 LLM의 답변을 보강합니다.
   - 적합한 사례: 사내 문서 기반 Q&A ("우리 회사 휴가 정책 알려줘", "제품 매뉴얼에서 설치 방법 찾아줘")

3. AI 에이전트 (Agent)
   - 정의: LLM이 도구를 사용하여 자율적으로 작업을 수행하는 시스템
   - 한 문장 설명: LLM이 '생각-행동-관찰' 사이클을 통해 외부 도구를 사용하여 목표를 달성합니다.
   - 적합한 사례: 복합 작업 자동화 ("오늘 날씨 확인하고 캘린더에 일정 추가해줘", "주가 검색해서 분석해줘")

핵심 차이점:
- LLM: 지식 -> 답변 (수동적)
- RAG: 검색 + 지식 -> 답변 (지식 확장)
- 에이전트: 추론 + 행동 + 도구 -> 작업 수행 (능동적)
"""

print(answer_q1)

**풀이 설명**

- **접근 방법**: 각 시스템의 핵심 특징(입력, 처리 방식, 출력)을 구분
- **핵심 개념**: RAG는 '지식 확장', 에이전트는 '행동 확장'
- **실무 팁**: 요구사항에 맞는 시스템 선택이 중요 (단순 Q&A vs 작업 자동화)

---

### Q2. ReAct 프레임워크 이해하기

**문제**: ReAct 프레임워크의 3단계(Thought, Action, Observation)를 설명하고, "내일 서울 날씨 알려줘"라는 질문에 대해 각 단계가 어떻게 진행될지 예시를 작성하세요.

In [ ]:
# 정답
answer_q2 = """
=== ReAct 프레임워크 설명 ===

ReAct = Reason (추론) + Act (행동)

3단계 설명:

1. Thought (생각)
   - LLM이 현재 상황을 분석하고 다음에 할 행동을 결정
   - "어떤 도구를 사용해야 할까?", "정보가 충분한가?" 등을 판단

2. Action (행동)
   - 결정한 도구를 실제로 실행
   - 도구 이름과 입력값을 지정하여 호출

3. Observation (관찰)
   - 도구 실행 결과를 받아서 확인
   - 이 결과를 바탕으로 다음 Thought 진행

---

예시: "내일 서울 날씨 알려줘"

[Round 1]
Thought: 사용자가 내일 서울 날씨를 물어봤어. 
         먼저 오늘 날짜를 확인하고, 날씨 API를 호출해야겠다.
         get_current_time 도구로 오늘 날짜를 먼저 확인하자.

Action: get_current_time()

Observation: 2025-01-29 14:30:00

[Round 2]
Thought: 오늘이 1월 29일이니까 내일은 1월 30일이네.
         이제 날씨 검색 도구로 서울의 1월 30일 날씨를 검색해야겠다.

Action: weather_search(location="서울", date="2025-01-30")

Observation: 서울 2025-01-30 예보: 맑음, 최고 5도, 최저 -3도, 미세먼지 보통

[Round 3]
Thought: 날씨 정보를 얻었어. 이제 사용자에게 친절하게 답변하면 돼.

Final Answer: 내일(1월 30일) 서울 날씨는 맑을 예정입니다.
              최고기온 5도, 최저기온 영하 3도이고, 
              미세먼지는 '보통' 수준입니다. 따뜻하게 입으세요!
"""

print(answer_q2)

**풀이 설명**

- **접근 방법**: 각 단계의 역할을 명확히 구분하고, 실제 시나리오로 적용
- **핵심 개념**: Thought는 '계획', Action은 '실행', Observation은 '결과 확인'
- **주의사항**: 한 번의 사이클로 끝나지 않고 여러 번 반복될 수 있음

---

### Q3. 도구 설명의 중요성

**문제**: 에이전트에서 도구의 `description`이 왜 중요한지 설명하고, 좋은 도구 설명의 3가지 조건을 서술하세요.

In [ ]:
# 정답
answer_q3 = """
=== 도구 설명(description)의 중요성 ===

왜 중요한가?

LLM은 도구의 '설명(description)'을 읽고 언제 어떤 도구를 사용할지 결정합니다.
- 설명이 명확하면 -> 올바른 도구 선택
- 설명이 모호하면 -> 잘못된 도구 선택 또는 도구 미사용

비유: 도구 설명은 '사용 설명서'와 같습니다.
      설명서가 부실하면 도구를 제대로 사용할 수 없습니다.

---

좋은 도구 설명의 3가지 조건:

1. 기능 명확성 (What)
   - 도구가 무엇을 하는지 명확히 설명
   - 좋은 예: "현재 한국 시간을 'YYYY-MM-DD HH:MM:SS' 형식으로 반환합니다."
   - 나쁜 예: "시간 관련 도구입니다."

2. 사용 상황 명시 (When)
   - 어떤 질문/상황에서 이 도구를 사용해야 하는지 안내
   - 좋은 예: "'지금 몇 시야?', '오늘 날짜가 뭐야?' 질문에 사용하세요."
   - 나쁜 예: (사용 상황 언급 없음)

3. 입출력 형식 안내 (How)
   - 입력 형식과 출력 형식을 명시
   - 좋은 예: "입력: 계산식 문자열 (예: '2 + 3 * 4'), 출력: 계산 결과"
   - 나쁜 예: "계산을 합니다."

---

실제 비교:

나쁜 설명:
  name: "calculator"
  description: "계산기"

좋은 설명:
  name: "calculator"
  description: "수학 계산을 수행합니다. 덧셈, 뺄셈, 곱셈, 나눗셈을 지원합니다.
                입력: 계산식 문자열 (예: '2 + 3 * 4')
                출력: 계산 결과
                '몇 곱하기 몇은?', '계산해줘' 등의 질문에 사용하세요."
"""

print(answer_q3)

**풀이 설명**

- **접근 방법**: LLM의 도구 선택 과정을 이해하고, 좋은 설명의 조건 도출
- **핵심 개념**: 도구 설명 = LLM이 읽는 사용 설명서
- **실무 팁**: 도구가 제대로 선택되지 않으면 description 먼저 점검

---

### Q4. 새로운 도구 만들기

**문제**: `@tool` 데코레이터를 사용하여 주어진 텍스트의 글자 수를 세는 `count_characters` 도구를 만들고 테스트하세요.

In [ ]:
# 정답

@tool
def count_characters(text: str) -> str:
    """
    주어진 텍스트의 글자 수를 셉니다.
    공백을 포함한 전체 글자 수와 공백을 제외한 글자 수를 함께 반환합니다.
    '글자 수가 몇 개야?', '몇 글자야?' 등의 질문에 사용하세요.
    
    입력: 분석할 텍스트 문자열
    출력: 글자 수 정보
    """
    total_chars = len(text)
    chars_no_space = len(text.replace(" ", ""))
    
    return f"입력 텍스트: '{text}'\n전체 글자 수: {total_chars}개\n공백 제외: {chars_no_space}개"

# 도구 정보 확인
print(f"도구 이름: {count_characters.name}")
print(f"도구 설명: {count_characters.description}")
print("\n" + "=" * 50)

# 테스트
test_texts = [
    "Hello World",
    "안녕하세요 파이썬",
    "AI Agent 구축하기"
]

print("\n테스트 결과:")
for text in test_texts:
    print(f"\n{count_characters.invoke(text)}")
    print("-" * 30)

**풀이 설명**

- **접근 방법**: @tool 데코레이터 사용, docstring으로 설명 작성
- **핵심 개념**: 입력 타입 명시 (str), 반환값도 str로 통일
- **추가 기능**: 공백 포함/제외 글자 수 모두 제공하여 유용성 증가
- **실무 팁**: 도구는 항상 문자열을 반환하는 것이 안전 (LLM이 처리하기 쉬움)

---

### Q5. Agent Executor 이해하기

**문제**: `AgentExecutor`의 역할을 설명하고, `verbose=True` 옵션이 어떤 정보를 보여주는지 서술하세요.

In [ ]:
# 정답
answer_q5 = """
=== AgentExecutor의 역할 ===

AgentExecutor란?
LLM, 도구, 프롬프트를 하나로 묶어 ReAct 사이클을 실행하는 '조율자'입니다.

역할:
1. 사용자 입력 받기
2. LLM에게 Thought(생각) 요청
3. LLM이 선택한 도구 실행 (Action)
4. 결과(Observation)를 LLM에게 전달
5. 최종 답변(Final Answer)까지 반복
6. 메모리 관리 (대화 기록 저장/로드)
7. 오류 처리 (파싱 에러 등)

비유: 에이전트가 '직원'이라면, AgentExecutor는 '매니저'입니다.
      매니저가 직원에게 일을 시키고, 결과를 확인하고, 다음 일을 지시합니다.

---

verbose=True 옵션이 보여주는 정보:

1. 체인 진입/종료
   > Entering new AgentExecutor chain...
   > Finished chain.

2. 도구 호출 정보 (Action)
   Invoking: `get_current_time` with `{}`
   Invoking: `calculator` with `{'expression': '2 + 3'}`

3. 도구 실행 결과 (Observation)
   2025-01-29 14:30:00
   2 + 3 = 5

4. 최종 답변
   현재 시간은 2025년 1월 29일 14시 30분입니다.

디버깅 활용:
- 어떤 도구가 선택되었는지 확인
- 도구 입력값이 올바른지 확인
- 도구 결과가 예상대로인지 확인
- 오류 발생 지점 파악
"""

print(answer_q5)

**풀이 설명**

- **접근 방법**: AgentExecutor의 내부 동작 과정을 단계별로 설명
- **핵심 개념**: AgentExecutor = ReAct 사이클의 실행기
- **실무 팁**: 개발 중에는 verbose=True, 프로덕션에서는 False

---

### Q6. 에이전트 프롬프트 분석하기

**문제**: 본문의 에이전트 프롬프트에서 `agent_scratchpad`와 `chat_history`가 각각 어떤 역할을 하는지 설명하세요.

In [ ]:
# 정답
answer_q6 = """
=== 에이전트 프롬프트의 핵심 변수 ===

1. agent_scratchpad (에이전트 메모장)
   
   역할:
   - 현재 ReAct 사이클 내에서의 중간 추론 과정을 저장
   - Thought, Action, Observation의 기록
   - 하나의 질문을 처리하는 동안만 유지됨
   
   예시:
   [Round 1]
   Thought: 날씨 도구를 사용해야겠다.
   Action: get_weather("서울")
   Observation: 맑음, 15도
   
   [Round 2]
   Thought: 정보를 얻었으니 답변을 생성하자.
   
   특징:
   - 질문마다 새로 시작 (초기화됨)
   - LLM이 이전 행동을 참고하여 다음 행동 결정

---

2. chat_history (대화 기록)
   
   역할:
   - 여러 질문에 걸친 대화의 맥락을 저장
   - 이전 질문과 답변의 기록
   - 세션 동안 지속됨 (Memory에 의해 관리)
   
   예시:
   User: 데이터브릭스가 뭐야?
   Assistant: 데이터브릭스는 데이터 및 AI 기업입니다.
   User: 그 회사가 최근에 발표한 게 뭐야?  <- '그 회사' = 데이터브릭스
   
   특징:
   - 여러 질문에 걸쳐 유지됨
   - 대명사 참조 해결 가능 ("그거", "그 회사" 등)
   - Memory 설정에 따라 저장 범위 결정 (k=5면 최근 5개)

---

비교 요약:

| 항목 | agent_scratchpad | chat_history |
|------|------------------|---------------|
| 범위 | 단일 질문 내 | 여러 질문 간 |
| 내용 | Thought/Action/Observation | 질문/답변 |
| 지속성 | 질문마다 초기화 | 세션 동안 유지 |
| 목적 | ReAct 사이클 추적 | 대화 맥락 유지 |
"""

print(answer_q6)

**풀이 설명**

- **접근 방법**: 두 변수의 범위와 목적을 비교하여 설명
- **핵심 개념**: scratchpad는 '작업 메모', history는 '대화 기록'
- **실무 팁**: 두 변수 모두 MessagesPlaceholder로 정의해야 함

---

### Q7. 메모리 옵션 비교하기

**문제**: `ConversationBufferWindowMemory`의 `k` 파라미터가 의미하는 바를 설명하고, `k`를 너무 크게/작게 설정했을 때의 장단점을 서술하세요.

In [ ]:
# 정답
answer_q7 = """
=== ConversationBufferWindowMemory의 k 파라미터 ===

k 파라미터란?
- 기억할 최근 대화의 수 (턴 수)
- k=5면 최근 5개의 (질문, 답변) 쌍을 기억
- 슬라이딩 윈도우 방식으로 오래된 대화는 자동 삭제

---

k를 너무 작게 설정했을 때 (예: k=1~2)

장점:
- 토큰 사용량 적음 -> 비용 절감
- 처리 속도 빠름
- 메모리 사용량 적음

단점:
- 맥락 유지 어려움 ("아까 그거"가 뭔지 모름)
- 복잡한 대화 흐름 처리 불가
- 사용자 경험 저하

적합한 경우:
- 단순 Q&A (한 번에 하나씩 독립적인 질문)
- 토큰 제한이 심한 모델

---

k를 너무 크게 설정했을 때 (예: k=50+)

장점:
- 긴 대화 맥락 유지 가능
- 복잡한 참조 해결 가능

단점:
- 토큰 사용량 급증 -> 비용 증가
- 처리 속도 느려짐
- 토큰 한도 초과 위험 (컨텍스트 윈도우 초과)
- 오래된 정보로 인한 혼란 가능

적합한 경우:
- 긴 상담/컨설팅 챗봇
- 프로젝트 진행 상황 추적

---

권장 설정:

| 사용 사례 | 권장 k값 |
|----------|----------|
| 간단한 Q&A 봇 | 3~5 |
| 일반 대화형 봇 | 5~10 |
| 상담/컨설팅 봇 | 10~20 |
| 프로젝트 관리 | 20~30 |

실무 팁:
- 대부분의 경우 k=5~10이 적절
- 사용자 피드백을 바탕으로 조정
- 토큰 비용과 사용자 경험의 균형 고려
"""

print(answer_q7)

**풀이 설명**

- **접근 방법**: k값의 의미를 이해하고, 극단적 설정의 영향 분석
- **핵심 개념**: k는 '맥락 유지'와 '비용/속도' 사이의 트레이드오프
- **실무 팁**: 사용 사례에 따라 적절한 k값 선택

---

### Q8. 커스텀 에이전트 구축하기

**문제**: 다음 두 도구를 사용하는 에이전트를 만들고 테스트하세요.

In [ ]:
# 정답

# 도구 1: 텍스트 뒤집기
@tool
def reverse_text(text: str) -> str:
    """
    주어진 텍스트를 뒤집어서 반환합니다.
    '뒤집어줘', '거꾸로', 'reverse' 등의 요청에 사용하세요.
    
    입력: 뒤집을 텍스트
    출력: 뒤집어진 텍스트
    """
    reversed_text = text[::-1]
    return f"'{text}'를 뒤집으면 '{reversed_text}'입니다."

# 도구 2: 단어 수 세기
@tool
def count_words(text: str) -> str:
    """
    주어진 텍스트의 단어 수를 셉니다.
    '단어가 몇 개야?', '몇 단어야?' 등의 질문에 사용하세요.
    
    입력: 분석할 텍스트
    출력: 단어 수
    """
    words = text.split()
    return f"'{text}'에는 {len(words)}개의 단어가 있습니다."

# 도구 리스트
custom_tools = [reverse_text, count_words]

print("도구 생성 완료!")
for t in custom_tools:
    print(f"  - {t.name}: {t.description[:50]}...")

In [ ]:
# 에이전트 구성
custom_prompt = ChatPromptTemplate.from_messages([
    ("system", """
당신은 텍스트 처리 전문가입니다.
사용자의 요청에 따라 적절한 도구를 선택하여 텍스트를 처리하세요.
한국어로 친절하게 답변하세요.
"""),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# 에이전트 생성
custom_agent = create_openai_tools_agent(llm, custom_tools, custom_prompt)
custom_executor = AgentExecutor(
    agent=custom_agent,
    tools=custom_tools,
    verbose=True
)

print("커스텀 에이전트 준비 완료!")

In [ ]:
# 테스트 1: 텍스트 뒤집기
response1 = custom_executor.invoke({"input": "Hello World를 뒤집으면 뭐야?"})
print("\n" + "=" * 50)
print(f"답변: {response1['output']}")

In [ ]:
# 테스트 2: 단어 수 세기
response2 = custom_executor.invoke({"input": "이 문장에 단어가 몇 개야: 파이썬은 재미있는 언어입니다"})
print("\n" + "=" * 50)
print(f"답변: {response2['output']}")

**풀이 설명**

- **접근 방법**: @tool로 도구 정의 -> 프롬프트 설정 -> 에이전트 조립 -> 테스트
- **핵심 개념**: 각 도구에 명확한 설명 작성이 중요
- **주의사항**: 도구 설명에 사용 상황을 명시해야 올바른 선택 유도

---

### Q9. 에이전트 디버깅

**문제**: 에이전트가 잘못된 도구를 선택하거나 예상과 다른 답변을 할 때, 문제를 진단하고 해결할 수 있는 방법 3가지를 서술하세요.

In [ ]:
# 정답
answer_q9 = """
=== 에이전트 디버깅 방법 ===

문제 상황:
- 잘못된 도구 선택 (날씨 물어봤는데 계산기 사용)
- 도구를 사용하지 않음 (직접 답변 생성)
- 예상과 다른 답변

---

방법 1: verbose=True로 추론 과정 확인

수행:
  agent_executor = AgentExecutor(..., verbose=True)

확인 사항:
- 어떤 Thought를 했는지
- 어떤 도구를 선택했는지
- 도구에 어떤 입력을 전달했는지
- Observation이 예상대로인지

해결:
- 선택 오류면 -> 도구 description 개선
- 입력 오류면 -> 도구 설명에 입력 형식 명시

---

방법 2: 도구 description 개선

문제 원인:
- 도구 설명이 모호함
- 사용 상황이 명시되지 않음
- 다른 도구와 역할이 겹침

개선 방법:
- 도구가 '무엇'을 하는지 명확히
- '언제' 사용해야 하는지 명시
- 입출력 형식 안내
- 다른 도구와 차별화

예시:
  Before: "날씨 도구"
  After: "실시간 날씨 정보를 조회합니다. '오늘 날씨', '내일 비 와?' 등의
          질문에 사용하세요. 입력: 도시 이름, 출력: 날씨 정보"

---

방법 3: 시스템 프롬프트 조정

문제 원인:
- 에이전트의 역할이 불명확
- 도구 사용 가이드라인 없음
- 우선순위 규칙 없음

개선 방법:
- 시스템 프롬프트에 도구 사용 가이드 추가
- 상황별 도구 선택 지침 명시
- 도구 사용 우선순위 설정

예시:
  "당신은 AI 비서입니다.
   도구 사용 가이드:
   - 날씨 관련 질문 -> weather_tool
   - 계산 관련 질문 -> calculator_tool
   - 일반 지식 질문 -> 도구 없이 답변
   
   항상 적절한 도구를 먼저 사용하세요."

---

추가 디버깅 팁:

1. 도구 개별 테스트
   - tool.invoke("test input")으로 각 도구 동작 확인

2. 간단한 질문부터 테스트
   - 복잡한 질문 전에 단순한 질문으로 동작 확인

3. temperature 조정
   - temperature=0으로 설정하여 일관된 동작 확보

4. 도구 수 제한
   - 너무 많은 도구는 선택 혼란 야기 (5~10개 권장)
"""

print(answer_q9)

**풀이 설명**

- **접근 방법**: 문제 발생 지점(도구 선택, 입력, 출력)을 파악하고 각각에 대한 해결책 제시
- **핵심 개념**: 대부분의 문제는 도구 description이나 시스템 프롬프트 개선으로 해결
- **실무 팁**: verbose=True로 항상 추론 과정 확인 후 디버깅

---

### Q10. 나만의 전문 에이전트 만들기

**문제**: 아래 주제 중 하나를 선택하여 최소 3개의 도구를 가진 에이전트를 구현하고, 3가지 질문으로 테스트하세요.

In [ ]:
# 정답 예시: 데이터 분석가 에이전트

# 도구 1: 기본 통계 계산
@tool
def calculate_statistics(numbers_str: str) -> str:
    """
    숫자 목록의 기본 통계(평균, 최대, 최소, 합계)를 계산합니다.
    '평균이 뭐야?', '최댓값은?', '통계 구해줘' 등의 질문에 사용하세요.
    
    입력: 쉼표로 구분된 숫자들 (예: "10, 20, 30, 40, 50")
    출력: 통계 결과
    """
    try:
        numbers = [float(n.strip()) for n in numbers_str.split(',')]
        stats = {
            "개수": len(numbers),
            "합계": sum(numbers),
            "평균": sum(numbers) / len(numbers),
            "최솟값": min(numbers),
            "최댓값": max(numbers)
        }
        result = f"입력: {numbers}\n"
        for key, value in stats.items():
            result += f"- {key}: {value:.2f}\n"
        return result
    except Exception as e:
        return f"오류: {str(e)}. 쉼표로 구분된 숫자를 입력해주세요."

# 도구 2: 텍스트 길이 분석
@tool
def analyze_text_length(text: str) -> str:
    """
    텍스트의 길이를 분석합니다 (글자 수, 단어 수, 문장 수).
    '몇 글자야?', '단어 수는?', '텍스트 분석해줘' 등의 질문에 사용하세요.
    
    입력: 분석할 텍스트
    출력: 길이 분석 결과
    """
    char_count = len(text)
    word_count = len(text.split())
    sentence_count = text.count('.') + text.count('!') + text.count('?')
    if sentence_count == 0:
        sentence_count = 1
    
    return f"""텍스트 분석 결과:
- 전체 글자 수: {char_count}개
- 공백 제외 글자 수: {len(text.replace(' ', ''))}개
- 단어 수: {word_count}개
- 문장 수: {sentence_count}개
- 평균 단어 길이: {char_count/word_count:.1f}자"""

# 도구 3: 텍스트 검색
@tool
def search_in_text(query: str) -> str:
    """
    저장된 데이터에서 키워드를 검색합니다.
    '~에 대해 찾아줘', '~가 뭐야?', '검색해줘' 등의 질문에 사용하세요.
    
    입력: 검색할 키워드
    출력: 검색 결과
    """
    # 샘플 데이터베이스
    database = {
        "파이썬": "파이썬은 1991년 귀도 반 로섬이 개발한 프로그래밍 언어로, 가독성이 좋고 배우기 쉽습니다.",
        "판다스": "판다스(pandas)는 데이터 분석을 위한 파이썬 라이브러리로, DataFrame이라는 자료구조를 제공합니다.",
        "머신러닝": "머신러닝은 데이터로부터 패턴을 학습하는 AI 기술로, 분류, 회귀, 클러스터링 등에 사용됩니다.",
        "딥러닝": "딥러닝은 인공신경망을 활용한 머신러닝 기법으로, 이미지, 자연어 처리에 뛰어난 성능을 보입니다."
    }
    
    query_lower = query.lower()
    for key, value in database.items():
        if key in query or query in key:
            return f"'{key}'에 대한 정보:\n{value}"
    
    return f"'{query}'에 대한 정보를 찾지 못했습니다. 검색 가능한 키워드: {list(database.keys())}"

# 도구 리스트
analyst_tools = [calculate_statistics, analyze_text_length, search_in_text, get_current_time]

print("데이터 분석가 도구 생성 완료!")
for t in analyst_tools:
    print(f"  - {t.name}")

In [ ]:
# 에이전트 구성
analyst_prompt = ChatPromptTemplate.from_messages([
    ("system", """
당신은 데이터 분석 전문가입니다.
사용자의 요청에 따라 적절한 도구를 선택하여 데이터를 분석하세요.

도구 사용 가이드:
- 숫자 통계 관련 -> calculate_statistics
- 텍스트 길이 분석 -> analyze_text_length
- 정보 검색 -> search_in_text
- 현재 시간 -> get_current_time

한국어로 친절하게 답변하세요.
"""),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# 메모리 설정
analyst_memory = ConversationBufferWindowMemory(
    k=5,
    memory_key="chat_history",
    return_messages=True
)

# 에이전트 생성
analyst_agent = create_openai_tools_agent(llm, analyst_tools, analyst_prompt)
analyst_executor = AgentExecutor(
    agent=analyst_agent,
    tools=analyst_tools,
    memory=analyst_memory,
    verbose=True,
    handle_parsing_errors=True
)

print("데이터 분석가 에이전트 준비 완료!")

In [ ]:
# 테스트 1: 통계 계산
response1 = analyst_executor.invoke({
    "input": "10, 20, 30, 40, 50의 평균과 최댓값 알려줘"
})
print("\n" + "=" * 50)
print(f"답변: {response1['output']}")

In [ ]:
# 테스트 2: 텍스트 분석
response2 = analyst_executor.invoke({
    "input": "이 문장을 분석해줘: 인공지능은 미래 기술의 핵심입니다. 많은 기업들이 AI에 투자하고 있습니다."
})
print("\n" + "=" * 50)
print(f"답변: {response2['output']}")

In [ ]:
# 테스트 3: 정보 검색
response3 = analyst_executor.invoke({
    "input": "판다스에 대해 알려줘"
})
print("\n" + "=" * 50)
print(f"답변: {response3['output']}")

**풀이 설명**

- **접근 방법**: 데이터 분석가 역할에 맞는 도구 3개 + 시간 도구 1개 구성
- **핵심 개념**: 각 도구에 명확한 역할 부여, 시스템 프롬프트에 도구 사용 가이드 추가
- **확장 방향**: 
  - 시각화 도구 추가 (차트 생성)
  - 파일 읽기 도구 추가
  - 외부 API 연동 (주가, 날씨 등)
- **실무 팁**: 도구 간 역할이 겹치지 않도록 명확히 구분

---

## 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 내용 | 실무 활용 |
|------|----------|----------|
| AI 에이전트 | LLM + 도구 + 추론 | 자율 작업 수행 AI |
| ReAct | Thought -> Action -> Observation | 단계적 문제 해결 |
| 도구 (Tool) | 외부 시스템과 상호작용 | API 호출, 검색 등 |
| @tool 데코레이터 | 함수를 도구로 변환 | 커스텀 도구 생성 |
| Agent Executor | ReAct 사이클 실행 | 에이전트 조율 |

### Part 2: 심화 핵심 요약

| 개념 | 핵심 내용 | 권장 설정 |
|------|----------|----------|
| 멀티-툴 에이전트 | 여러 도구 통합 | 도구 설명 명확히 |
| 메모리 | 대화 맥락 유지 | k=5~10 권장 |
| 도구 선택 | LLM이 설명 기반 결정 | 상세한 description |
| 에러 처리 | handle_parsing_errors | True 권장 |

### 에이전트 구현 체크리스트

```
1. 도구 준비:
   [x] 필요한 도구 목록 정의
   [x] @tool 데코레이터로 도구 생성
   [x] 도구 설명(description) 명확히 작성
   [x] 각 도구 개별 테스트
   
2. 에이전트 설정:
   [x] LLM 선택 (temperature=0 권장)
   [x] 프롬프트 템플릿 설계
   [x] 메모리 설정 (필요 시)
   [x] AgentExecutor 생성
   
3. 테스트:
   [x] 각 도구별 질문 테스트
   [x] 복합 질문 테스트
   [x] 메모리(대화 맥락) 테스트
   [x] 오류 상황 테스트
```

### 실무 팁

1. **도구 설명이 핵심**: LLM은 설명을 읽고 도구를 선택하므로 명확하게 작성
2. **verbose=True**: 개발 중에는 항상 켜서 추론 과정 확인
3. **temperature=0**: 에이전트는 일관된 행동이 중요하므로 낮게 설정
4. **에러 처리**: `handle_parsing_errors=True`로 안정성 확보
5. **도구 수 제한**: 너무 많은 도구는 선택 오류 증가 (5~10개 권장)